In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# AGS 068 – Knowledge Graph Completion using RotatE with FB15k-237

!pip install openke -q  # or use PyTorch-BigGraph or DGL-KE for newer systems

from openke.config import Trainer, Tester
from openke.module.model import RotatE
from openke.module.loss import SigmoidLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

# 🔧 DataLoader for FB15k-237
train_dataloader = TrainDataLoader(
    in_path = "./FB15k-237/", 
    nbatches = 100,
    threads = 8, 
    sampling_mode = "normal", 
    bern_flag = 1, 
    filter_flag = 1, 
    neg_ent = 25,
    neg_rel = 0
)

# 🧠 Define the RotatE Model
rotate = RotatE(
    ent_tot = train_dataloader.get_ent_tot(),
    rel_tot = train_dataloader.get_rel_tot(),
    dim = 512,
    margin = 6.0,
    epsilon = 2.0
)

model = NegativeSampling(model = rotate, loss = SigmoidLoss(), batch_size = train_dataloader.get_batch_size())

# 🚀 Train
trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 1000, alpha = 2e-5, use_gpu = True)
trainer.run()

# 🧪 Test
test_dataloader = TestDataLoader("./FB15k-237/", "link")
tester = Tester(model = rotate, data_loader = test_dataloader, use_gpu = True)
tester.run_link_prediction(type_constrain = False)

🧠 Example Query:

(Barack_Obama, ?, United_States) → Predict likely relations.

(Google, founded_by, ?) → Predict missing entity.

SyntaxError: invalid character '🧠' (U+1F9E0) (717374902.py, line 43)